## Run this set of cells once after restarting the kernel

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Must set SNORKELDB before importing SnorkelSession
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
session = SnorkelSession()

In [2]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 400
Sentences: 95656


In [3]:
from snorkel.models import candidate_subclass


#GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
PhenoPair = candidate_subclass('Phenotypes2',['descriptor', 'entity'])

In [8]:
#all_sents = set()
#docs = session.query(Document).order_by(Document.name).all()
#for i, doc in enumerate(docs):
    #for s in doc.sentences:
        #all_sents.add(s)
import cPickle
with open('small_data/pmcids_400.pkl', 'rb') as f:
    sent_dicts = cPickle.load(f)
train_ids, dev_ids, test_ids = set(sent_dicts['train']), set(sent_dicts['dev']), set(sent_dicts['test'])
all_ids = train_ids.union(dev_ids).union(test_ids)
# 40, 10, 10
train_sents, dev_sents, test_sents, all_sents = set(), set(), set(), set()
all_docs, train_docs, dev_docs, test_docs = set(), set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()

tair = []
with open('small_data/tair_labels.txt', 'rb') as f:
    for line in f:
        tair.append(line.strip())
tair = set(tair)

doc_sents = dict()
for doc_num, doc in enumerate(docs):
    #if len(all_docs) >=10: break
    #if len(train_docs) >= 40 and len(dev_docs) >= 10 and len(test_docs) >= 10:break
    doc_sents[doc_num] = set()
    for s in doc.sentences:
        doc_sents[doc_num].add(s)
        name = doc.name.split('-')[0]
        if name == 'PMC3503873':#in tair:
            all_docs.add(name)
            all_sents.add(s)
        if name in train_ids:
            train_docs.add(name)
            train_sents.add(s)
        elif name in dev_ids:
            dev_docs.add(name)
            dev_sents.add(s)
        elif name in test_ids:
            test_docs.add(name)
            test_sents.add(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))
    if name == 'PMC3503873': break

In [5]:
#all_sents = set()
#docs = session.query(Document).order_by(Document.name).all()
#for i, doc in enumerate(docs):
    #for s in doc.sentences:
        #all_sents.add(s)
import cPickle
with open('small_data/pmcids_400.pkl', 'rb') as f:
    sent_dicts = cPickle.load(f)
train_ids, dev_ids, test_ids = set(sent_dicts['train']), set(sent_dicts['dev']), set(sent_dicts['test'])
all_ids = train_ids.union(dev_ids).union(test_ids)
# 40, 10, 10
train_sents, dev_sents, test_sents, all_sents = set(), set(), set(), set()
all_docs, train_docs, dev_docs, test_docs = set(), set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()

tair = []
with open('small_data/tair_labels.txt', 'rb') as f:
    for line in f:
        tair.append(line.strip())
tair = set(tair)

doc_splits = {}
for doc_num, doc in enumerate(docs):
    name = doc.name.split('-')[0]
    if name in tair:
        
        doc_splits[name] = set()
        for s in doc.sentences:
            all_sents.add(s)
            doc_splits[name].add(s)

In [6]:
print len(all_sents)
for key in doc_splits.keys():
    print key, len(doc_splits[key])
#print all_sents

18852
PMC1557525 7
PMC5156837 155
PMC3504057 75
PMC2156172 285
PMC5384131 9
PMC2935865 108
PMC3002950 5
PMC2577656 8
PMC2935866 140
PMC4340836 237
PMC3139611 130
PMC2910046 126
PMC2905201 265
PMC3481204 130
PMC4938710 159
PMC3935568 220
PMC3040660 356
PMC2444027 137
PMC3879159 358
PMC2803217 243
PMC2323576 488
PMC3878228 155
PMC3017837 10
PMC3527283 352
PMC3542051 135
PMC2803208 104
PMC2893204 357
PMC1779593 7
PMC2877095 626
PMC3178619 187
PMC2607017 284
PMC2790454 6
PMC2566869 154
PMC3913338 524
PMC3883281 147
PMC2920754 120
PMC4283981 372
PMC2955748 370
PMC3503873 274
PMC5132069 200
PMC2557145 167
PMC2791118 170
PMC2736899 184
PMC4955218 8
PMC2295253 351
PMC2295252 160
PMC2791111 197
PMC3193000 241
PMC3022413 175
PMC2633612 282
PMC3504491 189
PMC2990762 878
PMC3043248 168
PMC113751 6
PMC2803206 78
PMC3481211 180
PMC3883298 196
PMC2478689 9
PMC4949643 236
PMC2785485 316
PMC3511161 329
PMC2999534 405
PMC2803491 9
PMC3914372 237
PMC2890954 11
PMC2265440 215
PMC3542049 145
PMC2908147 240

## Run the next four cells for Gene-Pheno Pair Extraction

### these can be skipped for phenotype

In [6]:
from candidate_extraction import PM, GM
from snorkel.candidates import Ngrams, CandidateExtractor


gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=15)

cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=True, nested_relations=False)

KeyboardInterrupt: 

In [95]:
print "Extracting Candidates..."

cand_extractor.apply(dev_sents, split=0)
dev_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==0).all()
print "Number of dev candidates:", len(dev_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 559


In [96]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(dev_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [97]:
sv

## Run these next four cells for Pheno extraction

In [8]:
#from revised_extraction import PM
from pheno_candidates import PATO, OBO
from snorkel.candidates import Ngrams, CandidateExtractor

pheno_ngrams = Ngrams(n_max=15)

pheno_extractor = CandidateExtractor(PhenoPair, 
                                    [pheno_ngrams, pheno_ngrams], [PATO, OBO],
                                    nested_relations=True, self_relations=True)

In [16]:
print "Extracting Candidates..."

pheno_extractor.apply(all_sents, split=0)
pheno_cands = session.query(PhenoPair).filter(PhenoPair.split==0).all()
print "Number of dev candidates:", len(pheno_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 402


In [17]:
from snorkel.viewer import SentenceNgramViewer
svp = SentenceNgramViewer(pheno_cands, session, annotator_name = 'gold', height=400)

<IPython.core.display.Javascript object>

In [18]:
svp

In [25]:
print "Extracting Candidates..."

pheno_extractor.apply(doc_splits['PMC5181575'], split=0)
pheno_cands = session.query(PhenoPair).filter(PhenoPair.split==0).all()
print "Number of dev candidates:", len(pheno_cands)


Extracting Candidates...


KeyError: 'PMC5181575'

In [23]:
from snorkel.viewer import SentenceNgramViewer
svp = SentenceNgramViewer(pheno_cands, session, annotator_name = 'gold', height=400)

<IPython.core.display.Javascript object>

In [24]:
svp